Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Side Chain Conformations and Dunbrack Energies](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.01-Side-Chain-Conformations-and-Dunbrack-Energies.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Protein Design with a Resfile and FastRelax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Packing-design-and-regional-relax.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Packing and Relax
Keywords: PackRotamersMover, FastRelax, MoveMapFactory, cartesian, ResidueSelector, NeighborhoodResidueSelector, CDRResidueSelector, TaskFactory, TaskOperation, InitializeFromCommandline, RestrictToRepacking, PreventRepackingRLT, OperateOnResidueSubsetOperation, SimpleMetric, SequenceMetric, clone()

## Overview
Here, you will learn how to optimize the side-chains of a protein.  In Rosetta, we call this `packing`.  We will use TaskFactories to control which residues are optimized. We will use this knowledge to refine a specific region of a protein.  In the next workshop, we will use what we learned here to begin designing proteins.

In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

## Imports

Before we begin, we must import some specific machinery from Rosetta.  Much of these tools are automatically imported when we do `from pyrosetta import *`, however, some are not. You should get into the habit of importing everything you need.  This will get you comfortable with the organization of Rosetta and make it easier to find tools that are beyond the scope of these workshops.

In [ ]:
#Python
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *

#Core Includes
from rosetta.core.kinematics import MoveMap
from rosetta.core.kinematics import FoldTree
from rosetta.core.pack.task import TaskFactory
from rosetta.core.pack.task import operation
from rosetta.core.simple_metrics import metrics
from rosetta.core.select import residue_selector as selections
from rosetta.core import select
from rosetta.core.select.movemap import *

#Protocol Includes
from rosetta.protocols import minimization_packing as pack_min
from rosetta.protocols import relax as rel
from rosetta.protocols.antibody.residue_selector import CDRResidueSelector
from rosetta.protocols.antibody import *
from rosetta.protocols.loops import *
from rosetta.protocols.relax import FastRelax


## Intitlialization 

Here, we will use command-line options to set the relax rounds to 2 instead of default 5 for speed of demo.  This is a bit tricky to do in code.
We also set the input antibody numbering scheme so that Rosetta understands the nomenclature of our antibody. 


Note that typically, we would add these options: `-ex1` and `-ex2` in order to increase the amount of rotamers
 available for packing, but this will slow us down for the demo, so we are keeping this out.

In [ ]:
init('-use_input_sc -input_ab_scheme AHo_Scheme -ignore_unrecognized_res \
     -ignore_zero_occupancy false -load_PDB_components false -relax:default_repeats 2 -no_fconfig')

## Import and copy pose

Begin by importing a pose.  Here, we are going to use an antibody from the PDB. 
Note that we use the clone function to copy the pose into `original_pose`. Using the equal sign will only make what is known as a shallow copy in python and anything we do to pose will be seen in original_pose.  The clone operation copies all the data into the original_pose and is the Rosetta equivalent of the python module, `copy.deep_copy`. 

In [ ]:
#Import a pose
pose = pose_from_pdb("inputs/2r0l_1_1.pdb")
original_pose = pose.clone()

## Setup a Normal TaskFactory

A `TaskFactory` is what we use to control packing specific residues in a pose. We first pass InitializeFromCommandLine which uses any of the options specified in the `pyrosetta.init()` function.

The `TaskFactory` is made up of a list of `TaskOperations`. These Taskops make up the bread and butter of controlling packing (and subsequently design). The taskops can be given to factory in any order.  

In Rosetta, ALL residues are set to both pack AND design by default. We use taskops to turn things off, like creating an ice sculpture.  Here, we turn design off by using the `RestrictToRepacking` operation.

In [ ]:
tf = TaskFactory()
tf.push_back(operation.InitializeFromCommandline())
tf.push_back(operation.RestrictToRepacking())

## Setup The Packer

Here, we setup the packer and pass the TaskFactory to it.  In general, if something can be packed, it will accept a TaskFactory.  Every time the protein is packed for a single round, the TaskFactory will generate what is called the PackerTask.  This object has all of the instructions needed for Rosetta to do packing and design.  Some TaskOperations can respond to environmental changes in the pose, such as neighboring residues and dynamically change at each packing step.  This is one of the reasons we use the TaskFactory machinery to setup packing instead of hacking the PackerTask itself as has been done in some of the earlier tutorials you may see on the web. 

In [ ]:
packer = pack_min.PackRotamersMover()
packer.task_factory(tf)

#Note that we are not passing a scorefunction here.  We will use the default, cmd-line scorefunction, 
# which is accessed through rosetta.core.scoring.get_score_function() and part of the packer.  We use use a scorefunction later. 

#Run the packer. (Note this may take a few minutes)
#Skip for tests
if not os.getenv("DEBUG"):
    packer.apply(pose)

#Dump the PDB
pose.dump_pdb('/outputs/2r0l_all_repack.pdb')

Lets compare the energies of the before and after pose. Any difference?

In [ ]:

scorefxn = get_score_function()
before = scorefxn.score(original_pose)


Check the sequence of the pose and the original pose using either the `SequenceMetric` 
 `rosetta.core.simple_metrics.metrics.SequenceMetric` (and it's calculate function) OR directly using the pose's `.sequence()` function. 
  Make sure that the poses have the same sequence - as packing is just design using single residue rotamers.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Regional Packing

Lets pack just a single CDR loop.  How do we do this?  Use a `TaskFactory` and a `TaskOperation`.  
  To make things easier, we will use a `ResidueSelector` to select the CDR. You will see more of this later.
  Briefly, a ResidueSelector returns a boolean vector that is the length of the pose. Each boolean is an indication of whether the residue was selected or not.  Because the vector is the same length of the pose, the index is the residue number.  There are many `ResidueSelectors` in rosetta, including ones for `AND` and `OR` operations to combine them.  Note where most of them come from above: `from rosetta.core.select import residue_selector as selections`

### Make selection using the CDRResidueSelector and NeighborhoodResidueSelector

We will use the NeighborhoodResidueSelector to pack the CDR loop and its surrounding neighbors, which defaults to 6 angstrums.  Each time we pack, the TaskFactory will be used to generate packing instructions (`PackerTask`) - and subsequently, the neighbors that we are packing will be updated each time to reflect this changing state of the pose.

In [ ]:
nbr_selector = selections.NeighborhoodResidueSelector()
cdr_selector = CDRResidueSelector()
cdr_selector.set_cdr(h1)

Note that h1 is what is called an enum.  It is a named integer.  This is better than passing around strings, 
 and you will start to see many of these around Rosetta. They were imported when we imported the antibody namespace at 
 the top of this workshop. Score terms such as fa_dun are also enums.

In [ ]:
nbr_selector.set_focus_selector(cdr_selector)
nbr_selector.set_include_focus_in_subset(True)

### Restrict to our selection

Lets turn off packing for everything but the H1 loop and its neighbors. By using a TF, every time packing is done,
  we regenerate the neighbors.  Using the NeighborhoodResidueSelector we actually use the pose energies object that
  has a list of neighbors instead of doing an N by N calculation each time!
  
In order to do this, we create what is known as a Residue Level Task Operation, or RLT, and then pass that into the `OperateOnResidueSubset`.  Use your tab completion to see how many RLTs there are.

In [ ]:
prevent_repacking_rlt = operation.PreventRepackingRLT()

In [ ]:
#True indicates here that we are flipping the selection.  So that we are turning off everything but the CDR and
#  its neighbors.

prevent_subset_repacking = operation.OperateOnResidueSubset(prevent_repacking_rlt, nbr_selector, True )


Lets check to see what residues have been selected as the CDR, and then the CDR and its neighbors.  We will use this to make sure our PackerTask is setup properly

In [ ]:
cdr_res = []
print("CDR")
for i in select.get_residue_set_from_subset(cdr_selector.apply(pose)):
    print(i)
    cdr_res.append(i)
    
print("\nCDR+Neighbors")
for i in select.get_residue_set_from_subset(nbr_selector.apply(pose)):
    if i in cdr_res:
        print(i,"CDR")
    else:
        print(i)

### Reset the pose

In [ ]:

pose = original_pose.clone()

tf.push_back(prevent_subset_repacking)

pack_cdrs_and_neighbors_tf = tf.clone()

packer.task_factory(tf)

Before we start, lets take a look at our PackerTask.  Are we designing anything?  Does this match our selection?

In [ ]:
print(tf.create_task_and_apply_taskoperations(pose))

Now lets Run the packer and dump the PDB.

Note how many rotamers were used and how many positions were done.  Now run it again.  Have those numbers changed?
 Why?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Design

Lets design this CDR.   We already almost everything we need created, so lets do this!!

Note, since we added the RestrictToRepacking TaskOperation to our original `TaskFactory`, we need to reset (clear) it. 

In [ ]:

pose = original_pose.clone()

tf.clear()
tf.push_back(operation.InitializeFromCommandline())
tf.push_back(prevent_subset_repacking)

#Turn off design of neighbors
nbr_selector2 = selections.NeighborhoodResidueSelector()
nbr_selector2.set_focus_selector(cdr_selector)
nbr_selector2.set_include_focus_in_subset(False)

restrict_to_repack = operation.RestrictToRepackingRLT()
prevent_nbr_design = operation.OperateOnResidueSubset(restrict_to_repack, nbr_selector2, False )
tf.push_back(prevent_nbr_design)



Once again, lets check to make sure we have created the correct TaskFactory. You should now see that the CDRs we selected are set to design into any of the amino acids. 

In [ ]:
print(tf.create_task_and_apply_taskoperations(pose))

### Set and Run

Set the new tf to the packer and run it. This will take a few minutes as for each CDR we packing rotamers for all 20 amino acids instead of just the native residue. Take note of how many rotamers are built compared to when we didn't do any design.

Dump the PDB and take a look. 

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Relax

Lets first relax the whole protein.  This will take some time, so after you run it, take a break and stretch!  As usual, you can find the output in the `expected_outputs` directory of this directory.


In [ ]:

pose = original_pose.clone()
fr = FastRelax()

#Here, we have to set a scorefunction or we segfault.  
#  Error checking is important, and protocols should use a default scorefunction. We will manage.

fr.set_scorefxn(scorefxn)

#Lets run this.  This takes a very long time, so we are going decrease the amount of minimization cycles we use.
# This is generally only recommended for cartesian, but we just want everything to run fast at the moment.
fr.max_iter(100)

#Run the code

#Dump the pdb and take a look.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Check the energy difference here once again.  Note how low it is compared to simply packing

delta = ?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Regional Relax

So that was fun, but what we really want is to optimize regions of the protein that we care about. 
  This can be tricky, so to start off, lets do the basic thing and see what happens. 
  
We will use a `MoveMapFactory`, which allows us to use `ResidueSelectors`.  
More information on the `MoveMapFactory` in terms of RosettaScripts, can be found here: https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/MoveMapFactories/MoveMapFactories-RosettaScripts

The default is to have everything OFF first, and turn specific things on.

In [ ]:

pose = original_pose.clone()
mmf = MoveMapFactory()

#mm_enable and mm_disable are Enums (numbered variables) that come when we import the MMF
mmf.add_bb_action(mm_enable, cdr_selector)
#mmf.add_chi_action(mm_enable, cdr_selector) We are taking this out for speed.

mm  = mmf.create_movemap_from_pose(pose)

Lets take a look at our `MoveMap` that the `MoveMapFactory` creates. Pass the movemap to `print()`  We should only have a few residues there.

Here, we only want to pack the region and neighbors.  NOT all 500 residues!

Lets use the cloned TaskFactory we used to repack the CDRs and neighbors, since we will be allowing the backbone of the CDRs to move.

In [ ]:
fr.set_movemap_factory(mmf)
fr.set_task_factory(pack_cdrs_and_neighbors_tf)

Run this, take a break and then lets see the results.
Dump the pose and take a look at it.  

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**What is wrong with this Pose???**


Okay, so we know whats wrong here, right?  Right?
The pose is moving too much where we don't want it. This is due to the FoldTree
  There are two fixes for this - modify the foldtree (using a Loop foldtree as you will see in later tutorials) or use cartesian-space refinement. 
  
  Lets do the easier of the two - cartesian!
  
 ### Regional Relax - Cartesian

In [ ]:

pose = original_pose.clone()
cart_sf = create_score_function("ref2015_cart")
mmf.set_cartesian(True)
fr.set_movemap_factory(mmf)
fr.set_scorefxn(cart_sf)
fr.cartesian(True)

#This is a general recommendation for cartesian minimization - it lowers the number of maximum cycles.
# More than this only increases time of protocol, but has little effect on energies/structure
fr.max_iter(200)

Run relax Lets dump the pose and take a look again.  Does this look better?   How are the energies?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Regional Relax - Classic

Now lets do the the classic way.  Were going to take advantage of the fact that we have an Antibody here.

We need to create our FoldTree and apply it to our pose before we minimize.  We are going to make an AntibodyInfo object that will allow us to do this easier.  Otherwise, we need to know the starting/end points of the CDR loop we are interested in.  If you are interested in Antibodies, there is an Antibody Workshop later as well that will use some of the tools you have learned here. 

We will also use the function `fold_tree_from_loops` that is part of the loop modeling framework - also covered in later workshops.  Rosetta can be difficult for sure, but the ability to use all of the different libraries in Rosetta can be a powerful tool. 

In [ ]:
 
pose = original_pose.clone()

ab_info = AntibodyInfo(pose)
ft = FoldTree()


start = ab_info.get_CDR_start(h1, pose)
stop =  ab_info.get_CDR_end(h1, pose)
cutpoint = (stop-start)/2+start
cdr_loop = Loop(start, stop, cutpoint)
cdr_loops = Loops()
cdr_loops.add_loop(cdr_loop)
                                
fold_tree_from_loops(pose, cdr_loops, ft)
pose.fold_tree(ft)
original_ft = pose.fold_tree()
add_cutpoint_variants(pose)

#Add chainbreak term so we don't get wacky stuff.  This term helps keep the peptide closed during bb movement.
scorefxn_ch = scorefxn
scorefxn_ch.set_weight(rosetta.core.scoring.chainbreak, 100)

#Setup our FastRelax again for dihedral-space.

mmf.set_cartesian(False)
fr.set_scorefxn(scorefxn_ch)
fr.cartesian(False)
fr.set_movemap_factory(mmf)
fr.max_iter(0) #Reset to default - if its 0, then we don't set it in the MinMover that FastRelax runs

#Run relax here

#Skip for tests
if not os.getenv("DEBUG"):
    fr.apply(pose)

#Reapply the original fold tree
pose.fold_tree(original_ft)

pose.dump_pdb('outputs/2r0l_dih_rel_H1.pdb')

How does the pose look?  Better/worse than cartesian? No difference? How do the energies compare?  Which is more general? 

## Conclusions

That should get you started with packing/relax! We even covered some advanced topics you will see later this week such as the TaskFactory and Residue Selectors!

Onto the next one!

<!--NAVIGATION-->
< [Side Chain Conformations and Dunbrack Energies](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.01-Side-Chain-Conformations-and-Dunbrack-Energies.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Protein Design with a Resfile and FastRelax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Packing-design-and-regional-relax.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>